In [1]:
%matplotlib inline

from asteria import config, source
from asteria import interactions
from asteria.neutrino import Flavor

from enum import Enum, EnumMeta, _EnumDict
from pathlib import Path

import astropy.units as u

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

import os
import sys

import tables
import pickle

In [2]:
conf = config.load_config('../../data/config/test.yaml')
ccsn = source.initialize(conf)

In [3]:
# Define neutrino energy spectrum
E_min = 0.1; E_max = 100.1; dE = 0.1;
Enu = np.arange(E_min, E_max, dE) * u.MeV

# Define post-bounce times at which to evaluate
t_min = -1; t_max = 15; dt = 0.0001;
time = np.arange(t_min, t_max, dt) * u.s

In [4]:
class _Binning(tables.IsDescription):
    start  = tables.Float64Col()   
    stop   = tables.Float64Col() 
    step   = tables.Float64Col()
    nSteps = tables.Int64Col()
    
class _Flavors(tables.IsDescription):
    nu_e     = tables.BoolCol(dflt=False)
    nu_e_bar = tables.BoolCol(dflt=False)
    nu_x     = tables.BoolCol(dflt=False)
    nu_x_bar = tables.BoolCol(dflt=False)

class _Interactions(tables.IsDescription):
    InvBetaTab      = tables.BoolCol(dflt=False)
    InvBetaPar      = tables.BoolCol(dflt=False)
    ElectronScatter = tables.BoolCol(dflt=False)
    Oxygen16CC      = tables.BoolCol(dflt=False)
    Oxygen16NC      = tables.BoolCol(dflt=False)
    Oxygen18        = tables.BoolCol(dflt=False)

1) Get path form configs, then alter `config.py` with new variable data for hdfs path

## Writing `h5` File from Scratch

In [5]:
vars(conf)

{'_value': {'name': 'IceCube fast supernova simulation',
  'base_path': '{ASTERIA}/data',
  'verbose': False,
  'detector': {'name': 'IceCube_IC-86',
   'geometry': {'max_height': 1900.0,
    'deepcore_string': 81,
    'table': {'format': 'ascii',
     'path': 'detector/Icecube_geometry.20110102.complete.txt'}},
   'effvol': {'deepcore_rel_eff': 1.35,
    'table': {'format': 'ascii',
     'path': 'detector/effectivevolume_benedikt_AHA_normalDoms.txt'}}},
  'source': {'name': 'Sukh_SFHo_z9.6',
   'model': 'Sukhbold 9.6Msun progenitor with SFHo equation of state',
   'progenitor': {'mass': '9.6 M_sun', 'distance': '10 kpc'},
   'table': {'format': 'fits', 'path': 'spectra/sukhbold-SFHo-z9.6.fits'}},
  'IO': {'table': {'format': 'h5',
    'path': 'processed/sukhbold-SFHo-z9.6.h5'}}},
 '_path': [],
 'abs_base_path': '/data/ASTERIA/data'}

In [6]:
def initialize(config):
    h5path = '/'.join([conf.abs_base_path, conf.IO.table.path])
    try:
        h5file = tables.open_file( filename=h5path, mode='w', 
                               title='Simulations of ASTERIA Source: '+ config.source.name)
    
        group_options = h5file.create_group('/', 'options', 'Requested Simulation Options' )
        table_tbins = h5file.create_table(group_options, 'Time',
                                          _Binning, 'Signature time binning [s]' )
        table_Ebins = h5file.create_table(group_options, 'Enu',
                                          _Binning, 'Neutrino spectrum energy binning [MeV]' )  
        table_Flavors = h5file.create_table(group_options, 'Flavors',
                                            _Flavors, 'Requested Neutrino Flavors' )
        table_Interactions = h5file.create_table(group_options, 'Interactions',
                                                 _Interactions, 'Requested Neutrino Interactions' )

        group_data = h5file.create_group("/", 'data', 'ASTERIA output')
        for flavor in Flavor:
            vlarray_flavor = h5file.create_vlarray(group_data, flavor.name,
                                                   tables.Float64Atom(shape=()),
                                                   'Flavor: '+flavor.name )
    except NameError as e:
        h5file.close()
        raise NameError(e)
        
    except ValueError as e:
        h5file.close()
        raise ValueError(e)

    h5file.close()
    
def saveSim(config, options, result):
    h5path = '/'.join([conf.abs_base_path, conf.IO.table.path])
    
    if not Path(h5path).exists():
        print('Creating file: '+ h5path)
        initialize(config)
        
    h5file = tables.open_file( filename=h5path, mode='a' )
    
    InteractionEnum, FlavorEnum, Enu, time = options
    
    
    
    table_Interactions = h5file.root.options.Interactions
    interactions = table_Interactions.row
    
    for item in InteractionEnum.included:
        interactions[item] = True
    for item in InteractionEnum.excluded:
        interactions[item] = False
        
    interactions.append()
    table_Interactions.flush()
    
    
    
    table_Flavors = h5file.root.options.Flavors
    flavors = table_Interactions.row
    
    for item in FlavorEnum.included:
        flavors[item] = True
    for item in FlavorEnum.excluded:
        flavors[item] = False
        
    flavors.append()
    table_Flavors.flush()
    
    
    table_Ebins = h5file.root.options.Enu
    eBins = table_Ebins.row
    eBins['start'] = Enu.min()
    eBins[ 'stop'] = Enu.max()
    eBins[ 'size'] = Enu.size
    eBins[ 'step'] = (Enu.max() - Enu.min())/(Enu.size-1)
    eBins.append()
    table_Ebins.flush()
    
    
    table_tbins = h5file.root.options.Time
    tBins = table_tbins.row
    tBins['start']  = time.min()
    tBins['stop']   = time.max()
    tBins['nSteps'] = time.size
    tBins['step']   = (time.max() - time.min())/(time.size-1)
    tBins.append()
    table_tbins.flush()



    for nu, flavor in enumerate(Flavor):  
        vlarray = h5file.root.data[ flavor.name ]
        vlarray.append( result[nu] )

    h5file.close()

In [7]:

saveSim(conf, 1, 1)

TypeError: 'int' object is not iterable

## Appending to existing `h5` File.

In [ ]:
try:
    h5file.close()
except NameError:
    print( "h5 File not open yet")    
finally:
    h5file = tables.open_file( filename=ofile, mode='a' )


table_Flavors = h5file.root.config.Flavors
nu = table_Flavors.row
for flavor in Flavor:
    nu[ flavor.name ] = True
    break
nu.append()
table_Flavors.flush()
requestedInteractions = {}

table_Interactions = h5file.root.config.Interactions
interactions = table_Interactions.row
interactions['InvBetaTab'] = True
interactions.append()
table_Interactions.flush()

table_tbins = h5file.root.config.Time
tBins = table_tbins.row
tBins['step']   = dt
tBins['start']  = t_min
tBins['stop']   = t_max - 1
tBins['nSteps'] = time.size
tBins.append()
table_tbins.flush()

table_Ebins = h5file.root.config.Enu
eBins = table_Ebins.row
eBins['start']  = E_min + 1
eBins['stop']   = E_max
eBins['step']   = dE
eBins['nSteps'] = Enu.size
eBins.append()
table_Ebins.flush()

for flavor in Flavor:  
    vlarray = h5file.root.E_per_V[ flavor.name ]
    vlarray.append( np.ones(3) )

    
print(  h5file )
h5file.close()

## Searching for matching criteria from `h5` file.

Potential problem with time binning, could lead to inefficient I/O if the file is large enough. My main concern is the time binning.

Pass dictionaries for io variables to speed things along (For the user).

In [ ]:
try: 
    h5file.close()
except NameError:
    print( "h5 File not open yet")    
finally:
    h5file = tables.open_file( filename=ofile, mode='r' )

    table_Time = h5file.root.config.Time
table_Enu  = h5file.root.config.Enu
table_Flavors = h5file.root.config.Flavors
table_Interactions = h5file.root.config.Interactions



pass_Time = [ row.nrow for row in table_Time.where('stop == 14') ]
pass_Enu  = [ row.nrow for row in table_Enu.where('start == 1.1') ]
pass_Flavors = [ row.nrow for row in table_Flavors.where('nu_e') ]
pass_Interactions = [ row.nrow for row in table_Interactions.where('InvBetaTab') ]
pass_all = list( set(pass_Time).intersection( pass_Enu, pass_Flavors, pass_Interactions) )


print("Rows that pass the Time cuts:", pass_Time)
print("Rows that pass the Enu cuts:", pass_Enu)
print("Rows that pass the flavor cuts:", pass_Flavors)
print("Rows that pass the Interactions cuts:", pass_Interactions)
print("Rows that pass all cuts:", pass_all)

In [ ]:
# Define neutrino energy spectrum
E_min = 0.1; E_max = 100.1; dE = 0.1;
Enu = np.arange(E_min, E_max, dE) * u.MeV

# Define post-bounce times at which to evaluate
t_min = -1; t_max = 15; dt = 0.0001;
time = np.arange(t_min, t_max, dt) * u.s

xs = interactions.ElectronScatter().cross_section(Flavor.nu_e, Enu).to(u.m**2).value
lep = interactions.ElectronScatter().mean_lepton_energy(Flavor.nu_e, Enu).to(u.m**2 * u.MeV).value
cut = (lep > 0) & (xs > 0)
quan = lep[cut]/xs[cut]
fig, ax = plt.subplots(1,1, figsize=(9,6))
ax.plot( Enu[cut], quan);
ax.grid()
ymin = np.min( quan )
ymax = np.max( quan )        
ax.set( ylim=(ymin/2, ymax*2) );


In [ ]:
try:
    h5file.close()
except NameError:
    print( "h5 File not open yet")
    
finally:
    
    h5file = tables.open_file( filename=ofile, mode='w', title='Generated from ASTERIA configuration file: test.yaml' )



nu = table_Flavors.row
for flavor in Flavor:
    nu[ flavor.name ] = True
nu.append()
table_Flavors.flush()

interactions = table_Interactions.row
interactions['InvBetaPar'] = True
interactions['ElectronScatter'] = True
interactions['Oxygen16CC'] = True
interactions['Oxygen16NC'] = True
interactions['Oxygen18'] = True
interactions.append()
table_Interactions.flush()

tBins = table_tbins.row
tBins['step']   = dt
tBins['start']  = t_min
tBins['stop']   = t_max
tBins['nSteps'] = time.size
tBins.append()
table_tbins.flush()

eBins = table_Ebins.row
eBins['start']  = E_min
eBins['stop']   = E_max
eBins['step']   = dE
eBins['nSteps'] = Enu.size
eBins.append()
table_Ebins.flush()

print(h5file)
h5file.close()

In [32]:
a = np.linspace(0,10, 11)
print(a)
b = np.arange(0.,11., 1)
print(b)

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]


In [33]:
(a.max() - a.min())/(a.size-1)

1.0

In [34]:
a.max()

10.0

In [35]:
a.min()

0.0

In [36]:
a.size

11

In [21]:
a

array([ 1.  ,  1.19,  1.38,  1.57,  1.76,  1.95,  2.14,  2.33,  2.52,
        2.71,  2.9 ,  3.09,  3.28,  3.47,  3.66,  3.85,  4.04,  4.23,
        4.42,  4.61,  4.8 ,  4.99,  5.18,  5.37,  5.56,  5.75,  5.94,
        6.13,  6.32,  6.51,  6.7 ,  6.89,  7.08,  7.27,  7.46,  7.65,
        7.84,  8.03,  8.22,  8.41,  8.6 ,  8.79,  8.98,  9.17,  9.36,
        9.55,  9.74,  9.93, 10.12, 10.31, 10.5 , 10.69, 10.88, 11.07,
       11.26, 11.45, 11.64, 11.83, 12.02, 12.21, 12.4 , 12.59, 12.78,
       12.97, 13.16, 13.35, 13.54, 13.73, 13.92, 14.11, 14.3 , 14.49,
       14.68, 14.87, 15.06, 15.25, 15.44, 15.63, 15.82, 16.01, 16.2 ,
       16.39, 16.58, 16.77, 16.96, 17.15, 17.34, 17.53, 17.72, 17.91,
       18.1 , 18.29, 18.48, 18.67, 18.86, 19.05, 19.24, 19.43, 19.62,
       19.81, 20.  ])